In [6]:
!pip install kafka-python

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [11]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from kafka import KafkaProducer
import json

In [12]:
producer = KafkaProducer(bootstrap_servers='54.70.155.149:9092') #Same port as your Kafka server

topic_name = "sample-data"

In [13]:
access_token = "1412669800707330048-vsa6dPAQQKpOzggTFKq2Xue2jO6U4W"
access_secret= "cDCOldC8YzFxEx9P8IW6oAHaWJ1pSU2vpSlzJJRvMhf8F"

consumer_key = "QpgMPt5vsM2Zfgkixm24eDQWD"
consumer_secret = "67wvFXE2qKiCwh7Mm7iSMfwwPT3Q74pdr7YvtWh6BUeg19IJpC"

In [14]:
class twitterAuth():
    """SET UP TWITTER AUTHENTICATION"""

    def authenticateTwitterApp(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)

        return auth


In [15]:
class TwitterStreamer():

    """SET UP STREAMER"""
    def __init__(self):
        self.twitterAuth = twitterAuth()

    def stream_tweets(self):
        while True:
            try:
                listener = TweetsListener() 
                auth = self.twitterAuth.authenticateTwitterApp()
                stream = Stream(auth, listener)
                stream.filter(track=["modi"], stall_warnings=True, languages= ["en"])
            except Exception as e:
                print(e)


In [16]:
#Create a StreamListener instance
class TweetsListener(StreamListener):
    
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end"
        out_data = '{ "tweet":"'+str(msg['extended_tweet']['full_text']).replace("\n","")+'","user":"'+str(msg['user']['screen_name'])+'", "tweet_id":"'+str(msg['id_str'])+'" }'
        print(out_data)
        producer.send(topic_name, str.encode(out_data))
      else:
        # add at the end of each tweet "t_end" 
        out_data = '{ "tweet":"'+str(msg['text']).replace("\n","")+'","user":"'+str(msg['user']['screen_name'])+'", "tweet_id":"'+str(msg['id_str'])+'" }'
        print(out_data)
        producer.send(topic_name, str.encode(out_data))
      return True

    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True


  def on_error(self, status):
    print(status)
    return True

  def on_exception(self, exception):
        print(exception)
        return

In [17]:
if __name__ == "__main__":
    TS = TwitterStreamer() 
    TS.stream_tweets()

new message
{ "tweet":"RT @INCIndia: One thing is crystal clear, 'achhe din aaye hai', but only for Modi &amp; friends, not for common Indians. https://t.co/EZMxwxjALn","user":"NadeemA99550374", "tweet_id":"1415263935180283907" }
new message
{ "tweet":"RT @INCIndia: One thing is crystal clear, 'achhe din aaye hai', but only for Modi &amp; friends, not for common Indians. https://t.co/EZMxwxjALn","user":"HakimSaifyINC", "tweet_id":"1415263936664850438" }
new message
{ "tweet":"RT @mvmeet: Tamil actor Vijay imported a Rolls Royce Ghost from EnglandHe wasn't ready to pay 'entry tax', so he filed petitionBut Madr…","user":"ManiShankarTri1", "tweet_id":"1415263938195824643" }
new message
{ "tweet":"RT @Vignesh_TMV: Difference between Oxford University and Whatsapp University. India is reeling under Modi Made DISASTERS ! #Resign_PRi…","user":"saraf_pankaj", "tweet_id":"1415263938950754304" }
new message
{ "tweet":"RT @TheShobhitAzad: Modi govt extended rebate on central taxes of textile prod

KeyboardInterrupt: 

In [11]:
# from kafka import KafkaConsumer
# import json

# topic_name = "tweet-data"

# consumer = KafkaConsumer(
#     topic_name,
#      bootstrap_servers=['localhost:9092'],
#      auto_offset_reset='earliest',
#      enable_auto_commit=True,
#      auto_commit_interval_ms =  5000,
#      fetch_max_bytes = 128,
#      max_poll_records = 100,

#      value_deserializer=lambda x: json.loads(x.decode('utf-8')))

# for message in consumer:
#  tweets = json.loads(json.dumps(message.value))
#  print(tweets)